In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [58]:
import warnings
warnings.filterwarnings("ignore")

In [59]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [60]:
df = pd.read_csv('../data/all_data/timeseries_data.csv')

In [61]:
df

,Unnamed: 0,Year,NOC,Athletes,Sports,Events,Gold,Silver,Bronze
0,0,1960,AFG,16,2,13,0,0,0
1,1,1960,AHO,5,1,4,0,0,0
2,2,1960,ARG,116,15,54,0,3,1
3,3,1960,AUS,277,19,125,10,21,11
4,4,1960,AUT,141,18,83,1,2,0
...,...,...,...,...,...,...,...,...,...
2696,2696,2016,VIE,28,10,24,1,1,0
2697,2697,2016,VIN,2,1,2,0,0,0
2698,2698,2016,YEM,3,3,3,0,0,0
2699,2699,2016,ZAM,6,3,6,0,0,0


In [62]:
df = df.drop(columns=['Unnamed: 0'])

In [63]:
df

,Year,NOC,Athletes,Sports,Events,Gold,Silver,Bronze
0,1960,AFG,16,2,13,0,0,0
1,1960,AHO,5,1,4,0,0,0
2,1960,ARG,116,15,54,0,3,1
3,1960,AUS,277,19,125,10,21,11
4,1960,AUT,141,18,83,1,2,0
...,...,...,...,...,...,...,...,...
2696,2016,VIE,28,10,24,1,1,0
2697,2016,VIN,2,1,2,0,0,0
2698,2016,YEM,3,3,3,0,0,0
2699,2016,ZAM,6,3,6,0,0,0


In [64]:
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.set_index('Year', inplace=True)

In [65]:
usa_data = df[df['NOC'] == 'USA']

In [66]:
usa_data['Total_Medals'] = usa_data['Gold'] + usa_data['Silver'] + usa_data['Bronze']

In [67]:
usa_data

,NOC,Athletes,Sports,Events,Gold,Silver,Bronze,Total_Medals
Year,,,,,,,,
1960-01-01,USA,474,23,164,97,24,22,143
1964-01-01,USA,619,29,190,96,39,42,177
1968-01-01,USA,664,28,199,100,37,36,173
1972-01-01,USA,719,31,216,72,77,46,195
1976-01-01,USA,660,29,225,73,61,40,174
1980-01-01,USA,144,10,38,24,4,1,29
1984-01-01,USA,842,35,255,190,121,50,361
1988-01-01,USA,896,37,274,89,67,58,214
1992-01-01,USA,955,40,303,94,57,86,237


In [68]:
usa_data = usa_data.drop(columns=['NOC'])

## Hisorical Athlete Participation

In [69]:
fig1 = go.Figure()

# Add traces for Athletes, Sports, Events, and Total Medals
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Athletes'], mode='lines+markers', name='Athletes', line=dict(color='blue')))
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Sports'], mode='lines+markers', name='Sports', line=dict(color='green')))
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Events'], mode='lines+markers', name='Events', line=dict(color='red')))
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Total_Medals'], mode='lines+markers', name='Total Medals', line=dict(color='purple')))

# Set title and labels
fig1.update_layout(
    title='Historical Athlete Participation for USA',
    xaxis_title='Year',
    yaxis_title='Count',
    legend_title='Metrics',
    template='plotly_white'
)

# Show the plot
fig1.show()


In [70]:
# Historical Medal Count
fig2 = go.Figure()

# Add traces for Gold, Silver, and Bronze Medals
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Gold'], mode='lines+markers', name='Gold Medals', line=dict(color='gold')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Silver'], mode='lines+markers', name='Silver Medals', line=dict(color='silver')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Bronze'], mode='lines+markers', name='Bronze Medals', line=dict(color='brown')))

# Set title and labels
fig2.update_layout(
    title='Historical Medal Count for USA',
    xaxis_title='Year',
    yaxis_title='Count',
    legend_title='Medal Type',
    template='plotly_white'
)

# Show the plot
fig2.show()


## Timeseries model

In [71]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [72]:
train = usa_data

In [73]:
# Fit VAR model
model = VAR(train)
model_fit = model.fit()

In [74]:
# Forecast for 2024
forecast = model_fit.forecast(train.values, steps=8)

In [75]:
# Print forecasted values for 2024
print("Forecasted values for 2024:")
years = pd.date_range(start='2017-01-01', end='2025-01-01', freq='Y')
forecast_df = pd.DataFrame(forecast, columns=usa_data.columns, index=years)
forecast_df.index = forecast_df.index.strftime('%Y-01-01')
forecast_df

Forecasted values for 2024:


,Athletes,Sports,Events,Gold,Silver,Bronze,Total_Medals
2017-01-01,430.487584,17.802324,103.667017,36.276874,48.234328,23.106314,107.617516
2018-01-01,908.690451,38.560508,310.278516,147.580334,82.552010,73.372615,303.504959
2019-01-01,306.846385,14.505271,71.807774,-3.684780,21.491604,15.465012,33.271836
2020-01-01,790.614145,33.956892,273.392515,149.603241,73.919040,64.005823,287.528103
2021-01-01,345.794983,15.698503,82.422330,13.152337,32.334963,17.701113,63.188414
2022-01-01,835.440370,35.616577,286.476088,149.416530,77.610850,66.802652,293.830032
2023-01-01,344.377987,15.788595,83.245385,8.601959,29.000508,17.860097,55.462565
2024-01-01,810.834014,34.699401,278.539631,146.819466,74.874182,64.948545,286.642193


## Visualizations for 2024 - USA

In [76]:
# Create traces for each plot
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Total_Medals'], mode='lines+markers', name='Total Medals'))
fig1.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Total_Medals'], mode='lines+markers', name='Total Medals'))
fig1.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Total_Medals'].iloc[-1]], mode='markers', name='2024 Forecast', marker=dict(color='red', size=14)))
fig1.update_layout(title='Total Medals Over the Years with 2024 Forecast',
                   xaxis_title='Year',
                   yaxis_title='Total Medals')
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Gold'], mode='lines+markers', name='Gold Medals', line=dict(color='gold')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Silver'], mode='lines+markers', name='Silver Medals', line=dict(color='silver')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Bronze'], mode='lines+markers', name='Bronze Medals', line=dict(color='#CD7F32')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Gold'], mode='lines+markers', name='Gold Medals', line=dict(color='gold')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Silver'], mode='lines+markers', name='Silver Medals', line=dict(color='silver')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Bronze'], mode='lines+markers', name='Bronze Medals', line=dict(color='#CD7F32')))
fig2.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Gold'].iloc[-1]], mode='markers', name='Gold 2024 Forecast', marker=dict(color='gold', size=14)))
fig2.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Silver'].iloc[-1]], mode='markers', name='Silver 2024 Forecast', marker=dict(color='silver', size=14)))
fig2.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Bronze'].iloc[-1]], mode='markers', name='Bronze 2024 Forecast', marker=dict(color='#CD7F32', size=14)))
fig2.update_layout(title='Medals (Gold, Silver, Bronze) Over the Years with 2024 Forecast',
                   xaxis_title='Year',
                   yaxis_title='Number of Medals')
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Athletes'], mode='lines+markers', name='Athletes'))
fig3.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Events'], mode='lines+markers', name='Events'))
fig3.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Sports'], mode='lines+markers', name='Sports'))
fig3.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Athletes'], mode='lines+markers', name='Athletes'))
fig3.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Events'], mode='lines+markers', name='Events'))
fig3.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Sports'], mode='lines+markers', name='Sports'))
fig3.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Athletes'].iloc[-1]], mode='markers', name='Athletes 2024 Forecast', marker=dict(color='blue', size=14)))
fig3.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Events'].iloc[-1]], mode='markers', name='Events 2024 Forecast', marker=dict(color='green', size=14)))
fig3.add_trace(go.Scatter(x=[forecast_df.index[-1]], y=[forecast_df['Sports'].iloc[-1]], mode='markers', name='Sports 2024 Forecast', marker=dict(color='orange', size=14)))
fig3.update_layout(title='Athletes, Events, and Sports Over the Years with 2024 Forecast',
                   xaxis_title='Year',
                   yaxis_title='Count')
fig3.show()


## Validation

In [77]:
medal_2021 = pd.read_excel('../data/2021/Medals.xlsx', engine='openpyxl')

In [78]:
medal_2021_usa = medal_2021[medal_2021['Team/NOC'] == 'United States of America']

In [79]:
medal_2021_usa

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1


In [80]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Total_Medals'], mode='lines+markers', name='Total Medals'))
fig1.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Total_Medals'], mode='lines+markers', name='Total Medals'))
fig1.add_trace(go.Scatter(x=['2021'], y=medal_2021_usa['Total'], mode='markers', name='2021 Actual', marker=dict(color='red', size=14)))
fig1.update_layout(title='Total Medals Over the Years with 2024 Forecast',
                   xaxis_title='Year',
                   yaxis_title='Total Medals')
fig1.show()


In [81]:

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Gold'], mode='lines+markers', name='Gold Medals', line=dict(color='gold')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Silver'], mode='lines+markers', name='Silver Medals', line=dict(color='silver')))
fig2.add_trace(go.Scatter(x=usa_data.index, y=usa_data['Bronze'], mode='lines+markers', name='Bronze Medals', line=dict(color='#CD7F32')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Gold'], mode='lines+markers', name='Gold Medals', line=dict(color='gold')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Silver'], mode='lines+markers', name='Silver Medals', line=dict(color='silver')))
fig2.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['Bronze'], mode='lines+markers', name='Bronze Medals', line=dict(color='#CD7F32')))
fig2.add_trace(go.Scatter(x=[2021], y=medal_2021_usa['Gold'], mode='markers', name='Gold 2024 Forecast', marker=dict(color='gold', size=14)))
fig2.add_trace(go.Scatter(x=[2021], y=medal_2021_usa['Silver'], mode='markers', name='Silver 2024 Forecast', marker=dict(color='silver', size=14)))
fig2.add_trace(go.Scatter(x=[2021], y= medal_2021_usa['Bronze'], mode='markers', name='Bronze 2024 Forecast', marker=dict(color='#CD7F32', size=14)))
fig2.update_layout(title='Medals (Gold, Silver, Bronze) Over the Years with 2024 Forecast',
                   xaxis_title='Year',
                   yaxis_title='Number of Medals')
fig2.show()

## Function

In [82]:
def forecast(country_data, country):
    # Historical data for the specified country
    country_data['Total_Medals'] = country_data['Gold'] + country_data['Silver'] + country_data['Bronze']
    country_data = country_data.drop(columns=['NOC'])
    
    # Fit VAR model
    model = VAR(country_data)
    model_fit = model.fit()
    
    # Forecast for 2024
    forecast = model_fit.forecast(country_data.values, steps=8)
    years = pd.date_range(start='2017-01-01', end='2025-01-01', freq='Y')
    forecast_df = pd.DataFrame(forecast, columns=country_data.columns, index=years)
    forecast_df.index = forecast_df.index.strftime('%Y-01-01')
    
    # Return forecast_df for saving to CSV
    return forecast_df

In [83]:
unique_countries = df['NOC'].unique()
forecasts = {}

# Iterate through each country, generate forecasts, and store in dictionary
for country in unique_countries:
    country_data = df[df['NOC'] == country]
    # skip if any error
    try:
        forecasts[country] = forecast(country_data, country)
    except:
        print(f"Error forecasting for {country}")

Error forecasting for MAL
Error forecasting for RHO
Error forecasting for UAR
Error forecasting for WIF
Error forecasting for SOM
Error forecasting for GAM
Error forecasting for YAR
Error forecasting for YMD
Error forecasting for EUN
Error forecasting for GBS
Error forecasting for SKN
Error forecasting for KIR
Error forecasting for TLS
Error forecasting for MHL
Error forecasting for KOS
Error forecasting for ROT
Error forecasting for SSD


In [84]:
# Save forecasts to CSV files for each country
for country, forecast_df in forecasts.items():
    filename = f'../data/all_data/forecasts/forecast_{country}.csv'
    forecast_df.to_csv(filename)

print("Forecasts saved to CSV files.")

Forecasts saved to CSV files.
